# QUANTUM ESPRESSO Example App

**Author: Aliaksandr Yakutovich (LSMO/THEOS, EPFL)**

This apps allows to compute energy, band structure or to optimize the geometry of a material with a minimal set of input parameters.

It is powered by:
- [Quantum ESPRESSO](https://www.quantum-espresso.org/) as the quantum engine
- [AiiDA](http://www.aiida.net) as the automation platform
- [AiiDA-QUANTUMESPRESSO](https://github.com/aiidateam/aiida-quantumespresso) plugin including custom-made workflows for AiiDA to manage the selection of parameters, the error handling, etc.

### Example steps to run:
1. SCF
1. Relaxation
1. Band structure calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os
import sys
from time import sleep 

orig = sys.stderr
sys.stderr = sys.stdout # redirect the stderr to stdout

In [ ]:
import ipywidgets as ipw
from aiidalab_widgets_base import aiidalab_display, CodQueryWidget, CodeDropdown, StructureUploadWidget
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.base import Str
from aiida.work.run import submit
from aiida.orm import load_node
from aiida.orm.utils import WorkflowFactory
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.parameter import ParameterData

In [ ]:
arrow_down = ipw.HTML("""<hr>
                         <br />
                         <center>
                         <i class="fa fa-arrow-down" style="color:#B0B0B0;font-size:12em;"></i>
                         </center>
                        """)
hr = ipw.HTML('<hr>')
run_btn = ipw.Button(description='Submit calculation')
code_group = CodeDropdown(input_plugin='quantumespresso.pw', text="Select code")
number_of_nodes = ipw.IntText(value=1,
                              step=1,description = "that will be run on",
                              disabled=False,
                              layout=ipw.Layout(width="180px"),
                              style={"description_width":"120px"},)

In [ ]:
structure_widget = StructureUploadWidget(
    examples=[
        ('Silicon', 'miscellaneous/structures/Si.xyz'),
        ('GaAs','miscellaneous/structures/GaAs.xyz')
    ],
    data_importers=[("COD", CodQueryWidget())],
    storable=False,
    node_class='StructureData')

def change_submit_calc_visibility(c):
    submit_out.layout.visibility = 'visible' if c['new'] else 'hidden'
structure_widget.observe(change_submit_calc_visibility, names=['has_structure'])

In [ ]:
def setup_calc():
    options =  {
        'max_wallclock_seconds': 3600*2,
        'resources':{
            'num_machines': number_of_nodes.value,
        }
    }
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([kpx.value, kpy.value, kpz.value])

    if code_group.selected_code is None:
        print ("Please select a code")
        return None

    parameters = {
        'SYSTEM': {
            'ecutwfc': 50.,
            'ecutrho': 200.,
        },
    }

    inputs = {
        'code': code_group.selected_code,
        'pseudo_family': Str(pseudo_family.value),
        'parameters': ParameterData(dict=parameters),
        'options': ParameterData(dict=options),
    }
    if 'optimized_structure' in globals():
        inputs['structure'] = optimized_structure
    else:
        inputs['structure'] = structure_widget.structure_node

    if run_type.label != 'bands':
        inputs['kpoints'] = kpoints
    return inputs

In [ ]:
def get_running_calculation(workchain):
    from aiida.orm import JobCalculation, WorkCalculation
    process = workchain

    while issubclass(type(process), WorkCalculation):
        try:
            for label, link in process.get_outputs_dict().items():
                if label.startswith('CALL_') and not link.is_sealed:
                    process = link
                    break
            else:
                return None
        except:
            return None

    if issubclass(type(process), JobCalculation):
        return process
    else:
        return None

In [ ]:
class ProgressBar(ipw.VBox):
    def __init__(self, process, **kwargs):
        self.process = process
        self.correspondance = {
            "NEW" : 0,
            "RUNNING": 1,
            "ABORTED": 2,
            "FAILED": 2,
            "FINISHED": 2,
        }
        self.bar = ipw.IntProgress(
            value=0,
            min=0,
            max=2,
            step=1,
            description='Progress:',
            bar_style='warning', # 'success', 'info', 'warning', 'danger' or ''
            orientation='horizontal',
            layout=ipw.Layout(width="800px")
        )
        self.state = ipw.HTML(description="Calculation state:", value='NEW', style={'description_width': 'initial'}) 
        children = [self.bar, self.state]
        super(ProgressBar, self).__init__(children=children, **kwargs)
    
    def update_state(self):
        self.bar.value = self.correspondance[self.current_state]
        if self.current_state == 'FINISHED':
            self.bar.bar_style = 'success'
        elif self.current_state in ["ABORTED", "FAILED"]:
            self.bar.bar_style = 'danger'
        else:
            self.bar.bar_style = 'info'
        self.state.value = self.current_state

    @property
    def current_state(self):
        attrs = self.process.get_attrs()
        if '_aborted' in attrs and attrs['_aborted']:
            return 'ABORTED'
        elif '_failed' in attrs and attrs['_failed']:
            return 'FAILED'
        elif '_finished' in attrs and attrs['_finished']:
            return 'FINISHED'
        else:
            return 'RUNNING'

In [ ]:
class ExportButton(ipw.Button):
    def __init__(self, process, **kwargs):
        self.process = process
        if 'description' not in kwargs:
            kwargs['description'] = "Export workflow ({})".format(self.process.pid)
        if 'layout' not in kwargs:
            kwargs['layout'] = {}
        kwargs['layout']['width'] = 'initial'
        super(ExportButton, self).__init__(**kwargs)
        self.on_click(self.export_aiida_subgraph)

    def export_aiida_subgraph(self, b=None):
        import base64
        import subprocess
        from tempfile import mkdtemp
        from IPython.display import Javascript
        fname = os.path.join(mkdtemp(), 'export.aiida')
        subprocess.call(['verdi', 'export', 'create', '-n', str(self.process.pid), fname])
        with open(fname, 'rb') as fobj:
            b64 = base64.b64encode(fobj.read())
            payload = b64.decode()
        js = Javascript(
            """
            var link = document.createElement('a');
            link.href = "data:;base64,{payload}"
            link.download = "{filename}"
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
            """.format(payload=payload,filename='export_{}.aiida'.format(self.process.pid))
        )
        display(js)

In [ ]:
def submit_process(b):
    global process_outputs
    output_text = ipw.Textarea(
        value="",
        placeholder="Calculation output will appear here",
        description='Calculation output:',
        layout={'width':"900px", 'height':'300px'},
        disabled=False,
        style = {'description_width': 'initial'}
    )
    inputs = setup_calc()
    if inputs is None:
        pass
    else:
        WorkChain = WorkflowFactory(run_type.value)
        process = submit(WorkChain, **inputs)
        process_node = load_node(process.pid)
        export_btn = ExportButton(process=process)
        process_bar = ProgressBar(process=process_node)
        display(export_btn, output_text, process_bar)
        previous_calc_pk = 0
        while not process_node.is_sealed:
            calc = get_running_calculation(process_node)
            try:
                if calc.pk != previous_calc_pk:
                    lines = []
                    previous_calc_pk = calc.pk
                    output_text.value = ''
                remote_path = calc.get_outputs_dict()['remote_folder'].get_remote_path()
                with open(os.path.join(remote_path, 'aiida.out')) as fobj:
                    lines += fobj.readlines()[len(lines):-1]
                    output_text.value = ''.join(lines)
            except:
                pass
            process_bar.update_state()
            sleep(0.1)
        process_bar.update_state() # updating the state for the last time to be 100% sure
        process_outputs = process_node.get_outputs_dict()
        output_text.layout={'visibility':'hidden'}
        if 'output_structure' in process_outputs:
            optimized_structure = process_outputs['output_structure']
            global optimized_structure
        display(arrow_down)
        show_results()

In [ ]:
def process_settings():
    global run_type, kpx, kpy, kpz, pseudo_family
    run_type = ipw.ToggleButtons(
        options=[
            ('scf', 'quantumespresso.pw.base'),
            ('relax', 'quantumespresso.pw.relax'),
            ('bands', 'quantumespresso.pw.bands'),
        ],
        description='Calculation type:',
        style = {'description_width': 'initial'},
    )
    kpx = ipw.BoundedIntText(value=2, min=1, step=1, description='k-points', layout=ipw.Layout(width="144px"))
    kpy = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpz = ipw.BoundedIntText(value=2, min=1, step=1, description='', layout=ipw.Layout(width="50px"))
    kpoints = ipw.HBox([kpx, kpy, kpz])
    pseudo_family = ipw.ToggleButtons(
        options = {
            'SSSP efficiency': 'SSSP_efficiency_v1.0',
            'SSSP accuracy': 'SSSP_precision_v1.0',
        },
        description='Pseudopotential family:',
        style = {'description_width': 'initial'},
    )

    display(arrow_down,
            ipw.HBox([code_group, number_of_nodes, ipw.HTML("node(s)")]),
            run_type,
            pseudo_family,
            kpoints,
            run_btn)

In [ ]:
def show_results():
    for key in ['output_parameters', 'output_structure', 'band_structure', 'retrieved']:
        if key in process_outputs:
            aiidalab_display(process_outputs[key])
    process_settings()

In [ ]:
submit_out = ipw.Output(layout={'visibility':'hidden'})
with submit_out:
    run_btn.on_click(submit_process)
    process_settings()
display(structure_widget, submit_out)